In [2]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

In [95]:
d        = 2           # Dimension of the function
n        = [ 20,  20]  # Shape of the tensor
r_matrix = 10

In [96]:
m        = 1.E+4                      # Number of calls to target function
nswp     = 50                         # Sweep number for ALS iterations
r        = 5                          # TT-rank of the initial random tensor

In [97]:
A = teneva.rand(n, r_matrix)
A = A[0][0] @ A[-1][:, :, 0]
A.shape

(20, 20)

In [98]:
def f(x): # BB
    return x @ A.T

def f_elem(i):
    x = np.zeros(n[1])
    x[i[1]] = 1.
    return f(x)[i[0]]

In [99]:
def func(I):
    y = []
    for i in I:
        a = f_elem(i) # A[tuple(i)]
        y.append(a)
    return np.array(y)

In [100]:
I_trn = teneva.sample_lhs(n, m, seed=42) 
y_trn = func(I_trn)

In [101]:
I_tst = teneva.sample_rand(n, 1.E+4, seed=42) 
y_tst = func(I_tst)

In [102]:
t = tpc()
Y = teneva.anova(I_trn, y_trn, r)
Y = teneva.als(I_trn, y_trn, Y, nswp)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :       0.34


In [103]:
Y = teneva.orthogonalize(Y)

And now we can check the result:

In [104]:
# Compute approximation in train points:
y_our = teneva.get_many(Y, I_trn)

# Accuracy of the result for train points:
e_trn = np.linalg.norm(y_our - y_trn)          
e_trn /= np.linalg.norm(y_trn)

# Compute approximation in test points:
y_our = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our - y_tst)          
e_tst /= np.linalg.norm(y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   3.42e-01
Error on test  :   3.54e-01


In [105]:
U = Y[0][0, :, :]
V = Y[1][:, :, 0]

In [106]:
np.max(np.abs(U @ V - A))

1.2666633865006947

In [107]:
U1, S1, V1 = np.linalg.svd(A, full_matrices=False)

In [109]:
S1[r:] = 0.
np.max(np.abs(U1 @ np.diag(S1) @ V1 - A))

1.2114543332388032

In [131]:
m         = 1.E+5  # Number of calls to target function
e         = None   # Desired accuracy
nswp      = None   # Sweep number
r         = 5      # TT-rank of the initial tensor
dr_min    = 0      # Cross parameter (minimum number of added rows)
dr_max    = 0      # Cross parameter (maximum number of added rows)

In [132]:
t = tpc()
info, cache = {}, {}
Y = teneva.rand(n, r)
Y = teneva.cross(func, Y, m, e, nswp, dr_min=dr_min, dr_max=dr_max,
    info=info, cache=cache)
Y = teneva.truncate(Y, 1.E-4) # We round the result at the end
t = tpc() - t

print(f'Build time           : {t:-10.2f}')
print(f'Evals func           : {info["m"]:-10d}')
print(f'Cache uses           : {info["m_cache"]:-10d}')
print(f'Iter accuracy        : {info["e"]:-10.2e}')
print(f'Sweep number         : {info["nswp"]:-10d}')
print(f'Stop condition       : {info["stop"]:>10}')
print(f'TT-rank of pure res  : {info["r"]:-10.1f}')
print(f'TT-rank of trunc res : {teneva.erank(Y):-10.1f}')

Build time           :       0.02
Evals func           :        270
Cache uses           :       1730
Iter accuracy        :   0.00e+00
Sweep number         :          5
Stop condition       :       conv
TT-rank of pure res  :        5.0
TT-rank of trunc res :        5.0


In [133]:
Y = teneva.orthogonalize(Y)

In [134]:
# Compute approximation in test points:
y_our = teneva.get_many(Y, I_tst)

# Accuracy of the result for test points:
e_tst = np.linalg.norm(y_our - y_tst) / np.linalg.norm(y_tst)

print(f'Error on test        : {e_tst:-10.2e}')

Error on test        :   5.82e-01


In [135]:
U = Y[0][0, :, :]
V = Y[1][:, :, 0]

In [136]:
np.max(np.abs(U @ V - A))

2.478295578380338